In [1]:
from pydantic import BaseModel, Field
from datasets import load_dataset, Dataset
from peft import LoraConfig

import xverify as xv

In [2]:

class Tools(BaseModel):
    """
    Run a tool.
    """
    tool_use: xv.XMLToolUse[xv.calculator, xv.search] = Field(..., description="The tool call to use")


class FinalAnswer(BaseModel):
    """
    Return a final answer.
    """
    answer: int = Field(..., description="Final answer to the question")


class Reason_and_Act(BaseModel):
    scratchpad: str = Field(
        ...,
        description="Information from the Observation useful to answer the question",
    )
    reasoning: str = Field(
        ...,
        description="It describes your thoughts about the question you have been asked",
    )
    response: Tools | FinalAnswer

def tool_response_func(model: Reason_and_Act) -> dict | None:
    return xv.run_tools(model.response)

guided_schema = xv.GuidedSchema(Reason_and_Act, schema="xml", tool_response_func=tool_response_func)

In [3]:

def format_prompt(prompt: str, system_prompt: str | None = None) -> list[dict[str, str]]:
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    return messages

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

SYSTEM_PROMPT = f"""\
You are a helpful assistant, responding in XML structured output.

- Think step by step using the scratchpad and reasoning outputs. You have {guided_schema.max_steps - 1} steps to think before responding.
- Use the tools provided. DO NOT rely on your own knowledge when a tool is available to help you.
- Respond with a final answer only once your are absolutely sure you have the answer.

Respond with a XML object, following the schema below:

{guided_schema.doc}
"""

CODE_PROMPT = """\
Given a math problem, use step-by-step reasoning and code execution to solve the problem.

For each step:
1. Think through your reasoning inside <reasoning> tags
2. Write Python scripts inside <code> tags to work out calculations
   - Functions and variables do not persist across <code> calls and should be redefined each time
   - Scripts should be written in Python 3.10+ syntax, and should run in under 10 seconds
   - Any desired outputs should be printed using print() statements
   - You may import numpy, scipy, and sympy libraries for your calculations
3. You will see the output from print() statements in your code in <output> tags
4. Continue until you can give the final answer inside <answer> tags
"""

dataset: Dataset = load_dataset("openai/gsm8k", "main", split="train") # type: ignore
dataset = dataset.map(lambda x: {
    "prompt": format_prompt(x["question"], CODE_PROMPT),
    "answer": extract_hash_answer(x["answer"])
})

In [4]:


model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model, tokenizer = xv.get_model_and_tokenizer(model_name)

run_name = "gsm8k-calculator-peft_" + model_name.split("/")[-1].lower()
training_args = xv.get_default_grpo_config(run_name, num_gpus=1)

peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj"]
)

def exact_answer_reward_func(completions, answer, **kwargs) -> list[float]:
    """Reward function that checks if the final answer matches the expected answer."""
    def _check_answer(trajectory: list[dict[str, str]], answer: int) -> float:
        """Extract the last answer from a trajectory."""
        last_message = trajectory[-1]
        assert last_message['role'] == 'assistant', "should be assistant"
        parsed: Reason_and_Act = guided_schema.parse(last_message['content']) # type: ignore
        if parsed is None or not isinstance(parsed.response, FinalAnswer):
            return 0.
        return 1. if parsed.response.answer == answer else 0.
    return [_check_answer(c, a) for c, a in zip(completions, answer)]

trainer = xv.GRPOGuidedTrainer(
    guided_schema=guided_schema,
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
    reward_funcs=[exact_answer_reward_func],
)
trainer.train()

Using Liger kernel
Applied Liger kernels to Qwen2


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/tom/fun/xverify/.venv/lib/python3.12/site-packages/trl/trainer/grpo_trainer.py:471: UserWarning: The requested device cuda:0 is also being used for training. For higher throughput and to avoid out-of-memory errors, it is recommended to use a dedicated device for vLLM. If this is intentional, you may ignore this warning but should adjust `vllm_gpu_memory_utilization` accordingly.
  warnings.warn(


INFO 03-13 10:35:40 __init__.py:207] Automatically detected platform cuda.
INFO 03-13 10:35:45 config.py:549] This model supports multiple tasks: {'embed', 'classify', 'score', 'reward', 'generate'}. Defaulting to 'generate'.
INFO 03-13 10:35:45 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-1.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_e

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-13 10:35:48 model_runner.py:1115] Loading model weights took 2.8875 GB
INFO 03-13 10:35:50 worker.py:267] Memory profiling takes 1.69 seconds
INFO 03-13 10:35:50 worker.py:267] the current vLLM instance can use total_gpu_memory (23.58GiB) x gpu_memory_utilization (0.30) = 7.07GiB
INFO 03-13 10:35:50 worker.py:267] model weights take 2.89GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 2.11GiB.
INFO 03-13 10:35:50 executor_base.py:111] # cuda blocks: 4929, # CPU blocks: 9362
INFO 03-13 10:35:50 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 2.41x
INFO 03-13 10:35:54 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utili

...

Validation error: 2 validation errors for Reason_and_Act
response.Tools
  Input should be a valid dictionary or instance of Tools [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
response.FinalAnswer
  Input should be a valid dictionary or instance of FinalAnswer [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
Validation error: 2 validation errors for Reason_and_Act
response.Tools
  Input should be a valid dictionary or instance of Tools [type=model_type, input_value='2000', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
response.FinalAnswer
  Input should be a valid dictionary or instance of FinalAnswer [type=model_type, input_value='2000', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type


╭──────────────────────────────────────────────────── Step 1 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ <|im_start|>system                              │ <Reason_and_Act>                                 │   0.00 │ │
│ │ Given a math problem, use step-by-step          │ <scratchpad>                                     │        │ │
│ │ reasoning and code execution to solve the       │ # Let's define the manufacturing years of the    │        │ │
│ │ problem.                                        │ three cars:                                      │        │ │
│ │                                                 │ # Car1 = 1970                                    │        │ │
│ │ For each step:                                  │ # Car2 = Car1 + 10 years  # 1970 + 10 = 1980     │        │ │
│ │ 1. Think through your reasoning inside          │ # Car3 = Car2 + 20 years  # 1980 + 20 = 2000     │        │ │
│ │ <reasoning> tags                                │ # This gives us the manufacturing year of the    │        │ │
│ │ 2. Write Python scripts inside <code> tags to   │ first car                                        │        │ │
│ │ work out calculations                           │ car1_year = 1970                                 │        │ │
│ │    - Functions and variables do not persist     │                                                  │        │ │
│ │ across <code> calls and should be redefined     │ # Calculate the year the second car was made     │        │ │
│ │ each time                                       │ car2_year = car1_year + 10                       │        │ │
│ │    - Scripts should be written in Python 3.10+  │                                                  │        │ │
│ │ syntax, and should run in under 10 seconds      │ # Calculate the year the third car was made      │        │ │
│ │    - Any desired outputs should be printed      │ car3_year = car2_year + 20                       │        │ │
│ │ using print() statements                        │                                                  │        │ │
│ │    - You may import numpy, scipy, and sympy     │ # Print the year the third car was made          │        │ │
│ │ libraries for your calculations                 │ print(car3_year)                                 │        │ │
│ │ 3. You will see the output from print()         │ </scratchpad>                                    │        │ │
│ │ statements in your code in <output> tags        │ <reasoning>                                      │        │ │
│ │ 4. Continue until you can give the final answer │ By adding 10 years to the year of the first car, │        │ │
│ │ inside <answer> tags                            │ we get its manufacturing year. Then, by adding   │        │ │
│ │ <|im_end|>                                      │ 20 years to that year, we get the manufacturing  │        │ │
│ │ <|im_start|>user                                │ year of the second car. Finally, we add 10 more  │        │ │
│ │ Three old cars displayed in an auction event    │ years to the manufacturing year of the second    │        │ │
│ │ have different manufacture dates. The first     │ car to get the manufacturing year of the third   │        │ │
│ │ car, made in 1970,  was made 10 years earlier   │ car.                                             │        │ │
│ │ than the second car. The third car was          │                                                  │        │ │
│ │ manufactured 20 years later after the second    │ </reasoning>                                     │        │ │
│ │ car was manufactured. Calculate the year that   │ <r

Validation error: 2 validation errors for Reason_and_Act
response.Tools
  Input should be a valid dictionary or instance of Tools [type=model_type, input_value="Let's write a Python scr...ld have paid on Sunday.", input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
response.FinalAnswer
  Input should be a valid dictionary or instance of FinalAnswer [type=model_type, input_value="Let's write a Python scr...ld have paid on Sunday.", input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
Validation error: 2 validation errors for Reason_and_Act
response.Tools
  Input should be a valid dictionary or instance of Tools [type=model_type, input_value='(18.0, 27.0)', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
response.FinalAnswer
  Input should be a valid dictionary or instance of FinalAnswer [type=model_type, input_value='(18.0, 27.0)', input_type=str]
    Fo

╭──────────────────────────────────────────────────── Step 1 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ <|im_start|>system                              │ <Reason_and_Act>                                 │   0.00 │ │
│ │ Given a math problem, use step-by-step          │ <scratchpad>                                     │        │ │
│ │ reasoning and code execution to solve the       │ 1. Reason: To determine how much Mario would     │        │ │
│ │ problem.                                        │ have paid on Sunday for his haircut, we need to  │        │ │
│ │                                                 │ calculate it 50% more than what he paid on       │        │ │
│ │ For each step:                                  │ Monday. If Monday's price is $18, then Sunday's  │        │ │
│ │ 1. Think through your reasoning inside          │ price would be $18 * 1.50 (to apply the 50%      │        │ │
│ │ <reasoning> tags                                │ increase). Thus, if $18 is the Saturday price,   │        │ │
│ │ 2. Write Python scripts inside <code> tags to   │ then $18 * 1.5 = ???                             │        │ │
│ │ work out calculations                           │ 2. Act:                                          │        │ │
│ │    - Functions and variables do not persist     │    - Calculate the price Mario would have paid   │        │ │
│ │ across <code> calls and should be redefined     │ on Sunday.                                       │        │ │
│ │ each time                                       │    - Use sympy library to define a symbol for    │        │ │
│ │    - Scripts should be written in Python 3.10+  │ the price on Monday and the variable             │        │ │
│ │ syntax, and should run in under 10 seconds      │ representing the price increase.                 │        │ │
│ │    - Any desired outputs should be printed      │    - Substitute Monday's price into the equation │        │ │
│ │ using print() statements                        │ and solve for the Sunday price.                  │        │ │
│ │    - You may import numpy, scipy, and sympy     │ </scratchpad>                                    │        │ │
│ │ libraries for your calculations                 │ <reasoning>                                      │        │ │
│ │ 3. You will see the output from print()         │ 1. Let's use the information provided to         │        │ │
│ │ statements in your code in <output> tags        │ calculate the price Mario would have paid for    │        │ │
│ │ 4. Continue until you can give the final answer │ his haircut on Sunday, which is 50% more         │        │ │
│ │ inside <answer> tags                            │ expensive than $18.                              │        │ │
│ │ <|im_end|>                                      │ 2. We'll use sympy to help us with the           │        │ │
│ │ <|im_start|>user                                │ calculations.                                    │        │ │
│ │ At Mario's barbershop haircuts are 50% more     │ </reasoning>                                     │        │ │
│ │ expensive during the weekends. If Mario paid    │ <response>                                       │        │ │
│ │ $18 for his last haircut on Monday, how much he │ Let's write a Python script to calculate the     │        │ │
│ │ would have paid the day before?<|im_end|>       │ price Mario would have paid on Sunday.           │        │ │
│ │ <|im_start|>assistant                           │ </response>                                      │        │ │
│ │                                                 │ </

ExpatError: no element found: line 21, column 5036